### Setup imports

In [16]:
!uv pip install torch torchvision

Using Python 3.12.8 environment at: /Users/jssrdrg/Documents/TU Delft/AE4S10/img_classification/.venv
Audited 2 packages in 3ms


In [17]:
import os
import torch
import numpy
from torchvision import datasets, transforms, models
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### Load datasets

In [9]:
DATASET_PATH = os.path.join(os.getcwd(), "../training_data/EuroSAT_RGB")

BATCH_SIZE = 10
TEST_SIZE = 0.2
EPOCHS = 10


In [10]:
transform = transforms.Compose([
  transforms.ToTensor()
])

In [11]:
dataset_full = datasets.ImageFolder(root=DATASET_PATH, transform=transform)

# Split into training and validation set
train_idx, val_idx = train_test_split(
  range(len(dataset_full)),       # Size of the full dataset
  test_size=TEST_SIZE,            # Fraction used for validation (test)
  stratify=dataset_full.targets   # Ensure class distribution is preserved
)

# Create the training and validation set
dataset_train = Subset(dataset_full, train_idx)
dataset_val = Subset(dataset_full, val_idx)

loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False)
loader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

# Show all the class labels
# print(dataset_full.classes)

### Set Model

In [12]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, len(dataset_full.classes)) # Adjust to the amount of classes

### Setup Training

In [13]:
# Set up loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cpu


### Model Training

In [14]:
# def train_model(model, loader_train, loader_val, criterion, optimizer):

for epoch in tqdm(range(EPOCHS), desc='Epochs'):

  # Training

  model.train()
  loss_train = 0
  correct = 0
  total = 0

  for inputs, labels in tqdm(loader_train, desc="Training"):
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    loss_train += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()

  train_acc = 100. * correct / total

  # Validation

  model.eval()
  loss_val = 0
  correct = 0
  total = 0

  # with torch.no_grad():

Epochs:   0%|          | 0/10 [06:47<?, ?it/s]


KeyboardInterrupt: 